# Pipeline test notebook

### Import requirements

In [152]:
%matplotlib ipympl

import glob
import cv2
import numpy
import matplotlib.pyplot as plt

from ImageProcessing import ImageProcessing
from StereoMatcher import StereoMatcher

### Loading images

In [153]:
path = "captures/testImages"
imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))
print ("Selections: 0-{}".format(len(imageGlobL)-1))

Selections: 0-5


### Select image pair

In [154]:
imageNumber = 5
imageL = cv2.imread(imageGlobL[imageNumber])
imageR = cv2.imread(imageGlobR[imageNumber])

### View images

In [155]:
cv2.imshow("left/right", numpy.hstack([imageL, imageR]))
cv2.waitKey(0)
cv2.destroyAllWindows()

### Load calibrations and other data

In [156]:
imageProcessing = ImageProcessing()
imageProcessing.loadMonoCalibrationResults()
imageProcessing.loadStereoCalibration()
imageProcessing.loadStereoRectify()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/stereoRectify.json
Loaded stereo rectification data


### Create stereo matcher

In [157]:
stereoMatcher = StereoMatcher("SGBM", \
                vertical=True, createRightMatcher=False)
stereoMatcher.referenceDispToDepthMatrix(\
imageProcessing.dispToDepthMatrix)

imageProcessing.initUndistortRectifyMap()

Reading from data/parametersSGBM.json


### Convert to grayscale and undistort

In [158]:
imageProcessing.convertToGrayscale(imageL, imageR)
imageProcessing.undistortRectifyRemap(imageProcessing.grayImageL, \
                                        imageProcessing.grayImageR)

### View results of above

In [159]:
cv2.imshow("left/right undistort", numpy.hstack(\
    [imageProcessing.undistortImageL, imageProcessing.undistortImageL]))

cv2.imshow("horizontal epipolar", \
    imageProcessing.drawHorEpipolarLines(\
        imageProcessing.undistortImageL, imageProcessing.undistortImageR))

cv2.imshow("vertical epipolar", \
    imageProcessing.drawVertEpipolarLines(\
        imageProcessing.undistortImageL, imageProcessing.undistortImageR))
        
cv2.waitKey(0)
cv2.destroyAllWindows()

### Compute disparity map

In [160]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessing.undistortImageL, \
                grayImageR=imageProcessing.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()
print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())

9.0
38.8125


### View disparity map

In [161]:
plt.figure()
plt.imshow(cv2.rotate(stereoMatcher.disparityMapL, \
    cv2.ROTATE_90_CLOCKWISE), cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute depth map

In [162]:
focalLength = imageProcessing.projectionMatrixL[0][0] # changes with rectify?
baseline = 32 # mm, measured irl

stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==0] = 0.9
stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==-1] = 0.9

depthMap = numpy.ones(stereoMatcher.disparityMapL.shape, numpy.single)
depthMap = (focalLength*baseline)/stereoMatcher.disparityMapL[:]

print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())
print (depthMap.shape)
print (depthMap.min())
print (depthMap.max())

9.0
38.8125
(640, 360)
610.7979
2634.066


### View depth map

In [163]:
plt.figure()
plt.imshow(cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE), cmap="Greys")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute point cloud

In [168]:
points = cv2.reprojectImageTo3D(stereoMatcher.disparityMapL, \
                                                    dispToDepthMatrix)

# Reshaping to a list of 3D coordinates
pointCloud = points.reshape((points.shape[0]*points.shape[1],3))[0::8]

print (points.shape)
print (pointCloud.shape)

print (points[0,0])
print (pointCloud[0])

(640, 360, 3)
(28800, 3)
[ -713.31146 -1168.7544   2763.0342 ]
[ -713.31146 -1168.7544   2763.0342 ]


Filtering extreme points

In [169]:
pointCloud = pointCloud[pointCloud[:, 2]>pointCloud[:, 2].min()]
pointCloud = pointCloud[pointCloud[:, 2]<pointCloud[:, 2].max()]
pointCloud = pointCloud[pointCloud[:, 1]>pointCloud[:, 1].min()]
pointCloud = pointCloud[pointCloud[:, 1]<pointCloud[:, 1].max()]
pointCloud = pointCloud[pointCloud[:, 0]>pointCloud[:, 0].min()]
pointCloud = pointCloud[pointCloud[:, 0]<pointCloud[:, 0].max()]

Rotate point cloud to have y forward

In [170]:
pointRotMatrix = numpy.array([  [0, 0, -1],
                                [-1, 0, 0],
                                [0, 1, 0]   ])

pointCloud = numpy.dot(pointCloud[:], pointRotMatrix)

print (pointCloud[0])

[309.37615967 731.39141846 141.42935181]


### View point cloud

In [171]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(pointCloud[:,0], pointCloud[:,1], pointCloud[:,2], s=1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at 0
ax.set_ylim(0,)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …